In [ ]:
import numpy as np
import itertools
import matplotlib as pltlib
table_arrangement = [[0,2], [1,3]]

test_preference_matrix = [[0,-10, 100, -100],
              [10, 0, 0, 100],
              [0,0,0,0],
              [-100, 100, -50, 0]] #ABCDABCD

In [3]:
def evaluate_table(table, matrix):
    score = 0
    for guest in table:
        for neighbor in table:
            if guest != neighbor:
                score += matrix[guest][neighbor]

    return score

def average_tables(tables, matrix):
    score = 0
    for table in tables:
        score += evaluate_table(table, matrix)
    average = score / len(tables)
    return average


ave = evaluate_table(table_arrangement, test_preference_matrix)
print(ave)

150.0
